## CS431/631 Data Intensive Distributed Analytics
### Winter 2022 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Yuhan Song
* **ID:** 20736900

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://www.student.cs.uwaterloo.ca/~cs451/assignment1-431.html).

We first need to install Apache Spark. Run the next block to download and install Spark. It will take about a minute to finish the instalation.

In [2]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located. The code in the following cell tells Python how to find this Spark installation. This code creates SparkContext (sc) for you. Do not change this block.

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster, and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Next, let's test that your `SparkContext` has been set up properly by running some simple test code.   This code uses a single Spark job to estimate the value of Euler's number $e$. One way to calculate $e$ is to use the following serries by Jacob Bernoulli:

$p_n = 1 - \frac{1}{1!} + \frac{1}{2!} - \frac{1}{3!} + \cdots + \frac{(-1)^n}{n!} = \sum_{k = 0}^n \frac{(-1)^k}{k!}$

As n tends to infinity, $p_n$ approaches $1/e$.

In the following code,  `parallelize()` and `map()` are Spark *transformations*, and `reduce()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take several seconds, since a Spark job is being created and executed, and should print an estimate of $e$ when it finishes.   


In [4]:
import math

n= 10000
inverse_e = sc.parallelize(range(0, n)).map(lambda x: ((-1)**x) * (1 / math.factorial(x))).reduce(lambda x,y:x+y)
e = 1 / inverse_e
print("e = ", e)

e =  2.718281828459044


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the example works.   What is the Spark job doing, and how is it used to estimate the value of $e$? How accurate is our estimate?

#### Your answer to Question 1:

According to the formula above, 1/e is approximated by summing ((-1)^k)/(k!) from k = 0 to k = n. By calling SparkContext’s parallelize method on an existing collection [0, 1, ... , 9999], elements of the collection are copied to form a distributed dataset that can be operated on in parallel. Map is a transformation that passes each element in the collection, x through the function ((-1)**x) * (1 / math.factorial(x)) and returns a new RDD representing the results. Lastly, the reduce action aggregates all the elements of the RDD using the sum function and returns the final result. Our estimate is accurate, but can be more accurate with the increase of n.





---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Run the next block to download the text file (Shakespeare.txt) and the Python tokenizer module (simple_tokenize.py).

In [5]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [6]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
  lines = sc.textFile("Shakespeare.txt")
  res = lines.flatMap(lambda x: simple_tokenize(x))
  return len(res.distinct().collect())

count_distinct_tokens()

25975

---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [7]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
  def process_line(t):
    tokens_list = []
    unique_t = list(set(t))
    n = len(unique_t)
    for i in range(0, n-1):
      for j in range(i+1, n):
        token1 = (unique_t[i], unique_t[j])
        token2 = (unique_t[j], unique_t[i])
        tokens_list += [token1, token2]
    return tokens_list
  lines = sc.textFile('Shakespeare.txt')
  res = lines.flatMap(lambda line: process_line(simple_tokenize(line)))
  return len(res.distinct().collect())

count_distinct_pairs()
    

1969760

---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [8]:
from simple_tokenize import simple_tokenize

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():
  lines = sc.textFile("Shakespeare.txt")
  num_of_lines = lines.count()
  res = lines.flatMap(lambda x: list(set(simple_tokenize(x)))).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
  res2 = res.map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[0] / num_of_lines, x[0], x[1]))
  return res2.take(50)

top_50_tokens_probabilities()

[(0.2009178657172255, 24604, 'and'),
 (0.19843538192686472, 24300, 'the'),
 (0.1523542765682928, 18657, 'i'),
 (0.148924529226347, 18237, 'to'),
 (0.1357526662202551, 16624, 'of'),
 (0.10844534452628657, 13280, 'a'),
 (0.09959332995802643, 12196, 'you'),
 (0.09430988583840991, 11549, 'my'),
 (0.08667461497003054, 10614, 'in'),
 (0.08630714204053634, 10569, 'that'),
 (0.07150206601447026, 8756, 'is'),
 (0.06720671577193814, 8230, 'not'),
 (0.06167012363422561, 7552, 'with'),
 (0.06039621747864574, 7396, 'me'),
 (0.05982459292165477, 7326, 'for'),
 (0.05836286726877787, 7147, 'it'),
 (0.054402325695340446, 6662, 'be'),
 (0.05246696826667102, 6425, 'this'),
 (0.05228731483447386, 6403, 'his'),
 (0.050899083767495794, 6233, 'your'),
 (0.05067043394469941, 6205, 'but'),
 (0.0474938346208496, 5816, 'he'),
 (0.046889545803459144, 5742, 'have'),
 (0.04132028940534714, 5060, 'thou'),
 (0.04015254209606559, 4917, 'as'),
 (0.03952375508337552, 4840, 'him'),
 (0.03949109082297604, 4836, 'so'),
 (0

---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

In [9]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
  def process_line(t):
    tokens_list = []
    unique_t = list(set(t))
    n = len(unique_t)
    for i in range(0, n-1):
      for j in range(i+1, n):
        token1 = (unique_t[i], unique_t[j])
        token2 = (unique_t[j], unique_t[i])
        tokens_list += [token1, token2]
    return tokens_list
  def cal_pmi(x, y, nxy, num_of_lines):
    px = x/num_of_lines
    py = y/num_of_lines
    pxy = nxy/num_of_lines
    return log(pxy/(px*py), 10)

  lines = sc.textFile('Shakespeare.txt')
  num_of_lines = lines.count()
  res_words = lines.flatMap(lambda line: list(set(simple_tokenize(line)))).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
  cnt = res_words.collectAsMap()
  res = lines.flatMap(lambda line: process_line(simple_tokenize(line))).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y).cache()
  res1 = res.filter(lambda x: x[1] >= threshold).map(lambda x: (x[0], cal_pmi(cnt[x[0][0]], cnt[x[0][1]], x[1], num_of_lines), x[1], cnt[x[0][0]], cnt[x[0][1]]))
  return res1.collect()

PMI(4)


[(('by', 'william'), 1.0352843433274252, 39, 3639, 121),
 (('william', 'by'), 1.0352843433274252, 39, 121, 3639),
 (('we', 'creatures'), 0.6154135352346697, 4, 3125, 38),
 (('creatures', 'we'), 0.6154135352346697, 4, 38, 3125),
 (('desire', 'from'), -0.06054994250143221, 4, 217, 2595),
 (('from', 'desire'), -0.06054994250143221, 4, 2595, 217),
 (('die', 'never'), 0.20417814877390827, 6, 461, 996),
 (('never', 'die'), 0.20417814877390827, 6, 996, 461),
 (('that', 'might'), 0.4799377756683868, 129, 10569, 495),
 (('might', 'that'), 0.4799377756683868, 129, 495, 10569),
 (('by', 'should'), 0.10808616191264653, 59, 3639, 1548),
 (('should', 'by'), 0.10808616191264653, 59, 1548, 3639),
 (('by', 'the'), 0.11866504544635662, 949, 3639, 24300),
 (('the', 'by'), 0.11866504544635662, 949, 24300, 3639),
 (('by', 'time'), 0.0952291027673476, 39, 3639, 1054),
 (('time', 'by'), 0.0952291027673476, 39, 1054, 3639),
 (('as', 'but'), 0.105972288063134, 318, 4917, 6205),
 (('but', 'as'), 0.1059722880631

---
#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```

Hint: Sampling must be done at the very last step. Hint: there is an action that returns a sample subset from an RDD.


In [18]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
  def process_line(t):
    tokens_list = []
    unique_t = list(set(t))
    n = len(unique_t)
    for i in range(0, n-1):
      for j in range(i+1, n):
        token1 = (unique_t[i], unique_t[j])
        token2 = (unique_t[j], unique_t[i])
        tokens_list += [token1, token2]
    return tokens_list
  def cal_pmi(x, y, nxy, num_of_lines):
    px = x/num_of_lines
    py = y/num_of_lines
    pxy = nxy/num_of_lines
    return log(pxy/(px*py), 10)

  lines = sc.textFile('Shakespeare.txt')
  num_of_lines = lines.count()
  res_words = lines.flatMap(lambda line: list(set(simple_tokenize(line)))).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
  cnt = res_words.collectAsMap()
  res = lines.flatMap(lambda line: process_line(simple_tokenize(line))).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y).cache()
  res1 = res.filter(lambda x: x[1] >= threshold).map(lambda x: (x[0][0], [(x[0], cal_pmi(cnt[x[0][0]], cnt[x[0][1]], x[1], num_of_lines), x[1], cnt[x[0][0]], cnt[x[0][1]])]))
  return res1.reduceByKey(lambda x, y: x+y).takeSample(False, samp_size)#.collect()#filter(lambda x: x[0] == "if" or x[0] == "you").collect()
PMI_one_token(3000,10)

[('the',
  [(('the', 'to'), 0.05123525982989819, 4072, 24300, 18237),
   (('the', 'and'), 0.0459349918330654, 5427, 24300, 24604),
   (('the', 'of'), 0.34294075191889295, 7266, 24300, 16624),
   (('the', 'i'), -0.08302344185158504, 3058, 24300, 18657)]),
 ('of',
  [(('of', 'the'), 0.34294075191889295, 7266, 16624, 24300),
   (('of', 'and'), 0.028305447826683594, 3565, 16624, 24604)]),
 ('i',
  [(('i', 'the'), -0.08302344185158504, 3058, 18657, 24300),
   (('i', 'and'), -0.05037403533805645, 3338, 18657, 24604),
   (('i', 'to'), 0.046852605922459045, 3095, 18657, 18237)]),
 ('to',
  [(('to', 'the'), 0.05123525982989819, 4072, 18237, 24300),
   (('to', 'and'), 0.017522588038697152, 3815, 18237, 24604),
   (('to', 'i'), 0.046852605922459045, 3095, 18237, 18657)]),
 ('and',
  [(('and', 'to'), 0.017522588038697152, 3815, 24604, 18237),
   (('and', 'the'), 0.0459349918330654, 5427, 24604, 24300),
   (('and', 'of'), 0.028305447826683594, 3565, 24604, 16624),
   (('and', 'i'), -0.0503740353380

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.